In [29]:
# %load mnist_loader.py
"""
mnist_loader
~~~~~~~~~~~~
A library to load the MNIST image data.  For details of the data
structures that are returned, see the doc strings for ``load_data``
and ``load_data_wrapper``.  In practice, ``load_data_wrapper`` is the
function usually called by our neural network code.
"""

#### Libraries
# Standard library
import pickle
import gzip

# Third-party libraries
import numpy as np

def load_data():
    """Return the MNIST data as a tuple containing the training data,
    the validation data, and the test data.
    The ``training_data`` is returned as a tuple with two entries.
    The first entry contains the actual training images.  This is a
    numpy ndarray with 50,000 entries.  Each entry is, in turn, a
    numpy ndarray with 784 values, representing the 28 * 28 = 784
    pixels in a single MNIST image.
    The second entry in the ``training_data`` tuple is a numpy ndarray
    containing 50,000 entries.  Those entries are just the digit
    values (0...9) for the corresponding images contained in the first
    entry of the tuple.
    The ``validation_data`` and ``test_data`` are similar, except
    each contains only 10,000 images.
    This is a nice data format, but for use in neural networks it's
    helpful to modify the format of the ``training_data`` a little.
    That's done in the wrapper function ``load_data_wrapper()``, see
    below.
    """
    f = gzip.open('./data/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    """Return a tuple containing ``(training_data, validation_data,
    test_data)``. Based on ``load_data``, but the format is more
    convenient for use in our implementation of neural networks.
    In particular, ``training_data`` is a list containing 50,000
    2-tuples ``(x, y)``.  ``x`` is a 784-dimensional numpy.ndarray
    containing the input image.  ``y`` is a 10-dimensional
    numpy.ndarray representing the unit vector corresponding to the
    correct digit for ``x``.
    ``validation_data`` and ``test_data`` are lists containing 10,000
    2-tuples ``(x, y)``.  In each case, ``x`` is a 784-dimensional
    numpy.ndarry containing the input image, and ``y`` is the
    corresponding classification, i.e., the digit values (integers)
    corresponding to ``x``.
    Obviously, this means we're using slightly different formats for
    the training data and the validation / test data.  These formats
    turn out to be the most convenient for use in our neural network
    code."""
    tr_d, va_d, te_d = load_data()
    training_inputs = [x for x in tr_d[0]]
    training_results = np.array([vectorized_result(y) for y in tr_d[1]])
    training_data = zip(training_inputs, training_results)
    validation_inputs = [x for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [x for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    e = np.zeros(10)
    e[j] = 1.0
    return e

In [28]:
import numpy as np
import random
import time

class L2(object):
    @staticmethod
    def reg_update(weights, vs, lamb, n ):
        weights = [(1-lamb/n) * w + v for w,v in zip(weights,vs)]
        return weights
class L1(object):
    @staticmethod
    def reg_update(weights, vs, lamb, n ):        
        weights = [w - ( (lamb/n) * np.sign(w)  ) + v for w,v in zip(weights,vs)]
        return weights


class Network4(object):
    def __init__(self,sizes):
        self.sizes = sizes
        self.num_layers = len(sizes)
        self.bias = [np.random.randn(x,1) for x in sizes[1:]]
        self.weights = [np.random.randn(x,y)/np.sqrt(y) for x,y in zip(sizes[1:],sizes[:-1])]
        # Momentum-based gradient descent
        self.v = [np.zeros(w.shape) for w in self.weights]
        
    
    def SGD(self,training_data,epochs,mini_batch_size,eta,
            co_eff = 0,reg_method=L2,lamb = 0,
            learning_schedule = (0,0,0),early_stopping_n = 0,
            test_data=None):
        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)
        
        # early stopping functionality:
        best_accuracy=1
        
        training_data = list(training_data)
        n = len(training_data)
        
        for i in range(epochs):
            start_time = time.time()
            
            random.shuffle(training_data)
            mini_batches = [training_data[x:x+mini_batch_size] for x in range(0,n,mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch,eta,co_eff,reg_method,lamb,n)
                
            end_time = time.time()
            print("time:{}".format(end_time-start_time))
            if test_data:
                accuracy = self.evaluate(test_data)
                print("Epoch {} : {} / {}".format(i,accuracy,n_test))
            else:
                print("Epoch {} complete".format(i))
            
            # Early stopping:
            
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                no_accuracy_change = 0
                #print("Early-stopping: Best so far {}".format(best_accuracy))
            else:
                no_accuracy_change += 1
            
            if early_stopping_n > 0:
                if (no_accuracy_change == early_stopping_n):
                    #print("Early-stopping: No accuracy change in last epochs: {}".format(early_stopping_n))
                    return 
            
            if learning_schedule[0]>0 and learning_schedule[2]>0:
                if (no_accuracy_change == learning_schedule):
                        eta = eta / learning_schedule[1]
                        no_accuracy_change = 0
                        learning_schedule[2] = learning_schedule[2] - 1
                if learning_schedule[2] < 0:
                    return
                
    def update_mini_batch(self,mini_batch,eta,co_eff,reg_method,lamb,n):
        nabla_w , nabla_b = self.back_pro(mini_batch)
        
#         print(nabla_w[-1].shape)
#         print(self.weights[-1].shape)
#         print(nabla_b[-1].shape)
#         print(self.bias[-1].shape)
#         x=input()

        # Momentum-based gradient descent
        self.v = [(co_eff * v) - (eta * nw) for v,nw in zip(self.v,nabla_w)]
        
        self.weights = reg_method.reg_update(self.weights, self.v, lamb, n )
        # self.weights = [(1-lamb/n) * w + v for w,v in zip(self.weights,self.v)]
        self.bias = [b - eta * nb for b,nb in zip(self.bias,nabla_b)]
        
    def back_pro(self,mini_batch):
        m = len(mini_batch)
        
        nabla_b = [np.zeros(b.shape) for b in self.bias]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        nabla_bs = [np.zeros((b.shape[0],m)) for b in self.bias]
#         nabla_ws = [np.zeros((w.shape[0],w.shape[1],n)) for w in self.weights]
        
        x,y = zip(*mini_batch)
        x=np.array(x).T
        y=np.array(y).T
        
        
        activation = x
        activations = [x]       
        zs = []
        
        # feedward propagation
        for w,b in zip(self.weights,self.bias):
            z = np.dot(w,activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
            
 
        nabla_bs[-1] = self.delta_L(y,activations[-1])
        nabla_b[-1] = np.sum(nabla_bs[-1],axis=1).reshape(self.bias[-1].shape) / m

        for l in range(2,self.num_layers):
            nabla_bs[-l] = np.dot(self.weights[-l+1].T,nabla_bs[-l+1]) * sigmoid_prime(zs[-l])
            nabla_b[-l] = np.sum(nabla_bs[-l],axis=1).reshape(self.bias[-l].shape) / m
        
        

        for l in range(1,self.num_layers):
            for i in range(m):
                nabla_b_i = nabla_bs[-l][:,i]
                nabla_b_i = nabla_b_i.reshape(len(nabla_b_i),1)
                activation_i = activations[-l-1][:,i]
                activation_i = activation_i.reshape(1,len(activation_i))
                             
                nabla_w[-l] = nabla_w[-l] + np.dot(nabla_b_i,activation_i)
                
            nabla_w[-l] = nabla_w[-l] / m
            
            
        return nabla_w,nabla_b
        
    def delta_L(self,y,activation_L):
        """delta_L for sigmod activation function with cross-entropy cost function"""
        return activation_L - y
    
    def feedward(self,a):
        for w,b in zip(self.weights,self.bias):
#             print(w.shape)
            a = sigmoid(np.dot(w,a)+b)
        return a
    def evaluate(self,x):
        a,y = zip(*x)
        a=np.array(a).T
        y=np.array(y)
        
#         print(a.shape)
#         print(y.shape)
        
        test_results = np.argmax(self.feedward(a),axis=0)
#         print(self.feedward(a).shape)
#         print(test_results.shape)
#         x=input()
        return sum(y == test_results)

    
def sigmoid(x):
    return 1 / (1+np.exp(-x))
def sigmoid_prime(x):
    return sigmoid(x) * (1-sigmoid(x))




In [32]:
training_data, validation_data, test_data = load_data_wrapper()
training_data = list(training_data)

net4 = Network4([784,30,10])
net4.SGD(training_data, epochs=100000, mini_batch_size=10, eta=1.5,
        co_eff=0.3,reg_method=L2,lamb=1.5,learning_schedule=(10,10,100),
        early_stopping_n=0, test_data=validation_data)

time:2.965529680252075
Epoch 0 : 9282 / 10000
time:2.918476104736328
Epoch 1 : 9379 / 10000
time:2.9372758865356445
Epoch 2 : 9412 / 10000
time:2.9603168964385986
Epoch 3 : 9409 / 10000


KeyboardInterrupt: 

In [26]:
array1 = np.array([[-1,2],[3,4]])
print(array1)

np.sum(array1,axis=1)

np.sign(array1)

[[-1  2]
 [ 3  4]]


array([[-1,  1],
       [ 1,  1]])